In [4]:
import hdbcli
from hdbcli import dbapi
print(hdbcli.__version__)

2.18.22


In [5]:
# conn = dbapi.connect(key='VDB_BETA')
conn = dbapi.connect(
    address='9f98i7c5-155d-4238-a9f9-d24dfc56df13.hana.prod-eu12.hanacloud.ondemand.com', 
    port='443', 
    user='D37BFE44L98A4ABC81FFA9BD2B2C0A44_7CHYYW0NWAQ6KZCXJAS53KWV5_RT', 
    password='Ho28ijy-olsyb1XrVgsMsx59XPQcEbefMkSlCptzk3uymHW671JM2nXwaRIx0SxYeBHSgxNqP2AUGnGIacsBn-A9c6uMRGsYEng2gAgApqCjjb49p9GyGFxXGlHQH3tW', 
    encrypt=True
)

In [18]:
# DDL
cursor = conn.cursor()
sql_command = '''CREATE TABLE MYVECTORS(ID BIGINT, TEXT NCLOB, VECTOR REAL_VECTOR(2));'''
cursor.execute(sql_command)
cursor.close()

In [19]:
rows = []
rows.append([1, 'T1', '[0, 1]'])
rows.append([2, 'T2', '[0, 1]'])
rows.append([3, 'T3', '[0.1, 0.8]'])

In [20]:
# bulk insert
conn.setautocommit(False)
cursor = conn.cursor()
sql = 'INSERT INTO MYVECTORS (ID, TEXT, VECTOR) VALUES (?, ?, TO_REAL_VECTOR(?))'
try:
    cursor.executemany(sql, rows)
except Exception as e:
    conn.rollback()
    print("An error occurred:", e)
try:
    conn.commit()
finally:
    cursor.close()

In [21]:
# query
cursor = conn.cursor()
sql_command = '''SELECT ID, TEXT, TO_NVARCHAR(VECTOR) AS VECTOR_STR FROM MYVECTORS LIMIT 5;'''
cursor.execute(sql_command)

rows = cursor.fetchall()
for row in rows:
    print(row)
cursor.close()

(1, 'T1', '[0,1]')
(2, 'T2', '[0,1]')
(3, 'T3', '[0.1,0.8]')


In [27]:
# TOP k NN query
cursor = conn.cursor()
# cosine similarity DESC order
sql_command = '''SELECT ID, TEXT, COSINE_SIMILARITY(VECTOR, TO_REAL_VECTOR('[0.1, 0.9]')) AS COS_SIM FROM MYVECTORS 
ORDER BY COS_SIM DESC LIMIT 5;'''

# l2 distance ASC order
# sql_command = '''SELECT ID, TEXT, L2DISTANCE(VECTOR, TO_REAL_VECTOR('[0.1, 0.9]')) AS L2_DIST FROM MYVECTORS 
# ORDER BY L2_DIST ASC LIMIT 5;'''

# similarity matrix
# sql_command = '''SELECT V1.ID, V2.ID, COSINE_SIMILARITY(V1.VECTOR, V2.VECTOR) AS COS_SIM 
# FROM MYVECTORS AS V1, VDB.MYVECTORS AS V2 WHERE V1.ID < V2.ID ORDER BY COS_SIM DESC LIMIT 5;'''
cursor.execute(sql_command)

rows = cursor.fetchall()
for row in rows:
    print(row)

cursor.close()

(3, 'T3', 0.9999061386085962)
(1, 'T1', 0.9938837488013375)
(2, 'T2', 0.9938837488013375)
